In [ ]:
import radiomics
from radiomics import firstorder, glcm, ngtdm, shape, imageoperations, getFeatureClasses
import os, glob, shutil, tempfile
import numpy as np
import torch
import monai
import matplotlib.pyplot as plt

from monai.config import print_config
from monai.data import ArrayDataset, GridPatchDataset, PatchIter
from monai.transforms import (
    Compose,
    LoadImage,
    RandSpatialCrop,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import first
print_config()

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir) # these seem to be for downloaded test images. Does not apply to me.

In [ ]:
import sys
print(sys.maxsize > 2**32)

In [ ]:
!pwd

In [ ]:
!ls Data/21102002

In [ ]:
path = "./Data/"
temp_list = os.listdir(path)
cmb_list = []
orig_list = []
t1_list = []
for i, name in enumerate(temp_list):
    if "." in  name:
        temp_list.pop(i)
    else:
        cmb_list.append(path+name+"/cmb.nii.gz")
        orig_list.append(path+name+"/swi.nii.gz")
        t1_list.append(path+name+"/t1.nii.gz")

In [ ]:
orig_test = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(orig_list[0])
cmb_test = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(cmb_list[0])
t1_test = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(t1_list[0])
# it's a tensor

In [ ]:
print(f'swi shape:  {orig_test.shape},\ncmb shape:  {cmb_test.shape}')
print(f't1 shape: {t1_test.shape}')

In [ ]:
import logging
import SimpleITK as sitk
logger = radiomics.logger
logger.setLevel(logging.DEBUG)
handler = logging.FileHandler(filename="testLog.txt", mode="w")
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [ ]:
settings = {
    "binWidth":30,
    "minimumROIDimensions":2,
    "minumumROISize":None,
    "normalize":True, # weighted images = True
    "normalizeScale":100,
    "removeOutliers":None,
    "resampledPixelSpacing":[2, 2, 0], # for sampling
    "interpolator":sitk.sitkBSpline, # for sampling
    "preCrop":True,
    "padDistance":10,
    "label":1,
    "additionalInfo":True,
    "voxelArrayShift":300
}

<hr style="color:green">
<hr style="color:green">
SET THE INDIVIDUAL FEATURE SEPARATELY
<hr style="color:green">
<hr style="color:green">

In [ ]:
import six
orig_itk = sitk.ReadImage(orig_list[0])
cmb_itk = sitk.ReadImage(cmb_list[0])

In [ ]:
label_stats =sitk.LabelStatisticsImageFilter()
label_stats.Execute(image=orig_itk, labelImage=cmb_itk)
num_of_labels = label_stats.GetNumberOfLabels()
print(num_of_labels)

In [ ]:
orig_test[0].shape

In [ ]:
firstOrderFeatures = firstorder.RadiomicsFirstOrder(orig_itk, cmb_itk, **settings)
firstOrderFeatures.disableAllFeatures() # for resetting the instance
firstOrderFeatures.enableFeatureByName("Mean", True)
firstOrderFeatures.enableFeatureByName("Kurtosis", True)
firstOrderFeatures.enableFeatureByName("Median", True)
# firstOrderFeatures.enableFeatureByName("StandardDeviation", True)
firstOrderFeatures.enableFeatureByName("Skewness", True)
firstOrderFeatures.enableFeatureByName("Variance", True)

print("following first order features will be calculated")
for f in firstOrderFeatures.enabledFeatures.keys():
    print("  ", f)
    print(getattr(firstOrderFeatures, "get%sFeatureValue" % f).__doc__)
print("\nCalculating FO features!")
fo_result = firstOrderFeatures.execute()
print("Done!")

In [ ]:
# TEST CELL
# firstOrderFeatures.disableAllFeatures()
# firstOrderFeatures.enableAllFeatures()

In [ ]:
# check what's inside! It's Christmas! -not! .... !Christmas XD
for (key, val) in six.iteritems(fo_result):
    print(' ', key, ':', val)

In [ ]:
shapeFeatures = shape.RadiomicsShape(orig_itk, cmb_itk, **settings)
shapeFeatures.enableAllFeatures()

for f in shapeFeatures.enabledFeatures.keys():
    print(' ', f)
    print(getattr(shapeFeatures, 'get%sFeatureValue' % f).__doc__)
print("\n Calculating shape features...")
sh_feature = shapeFeatures.execute()
print("done!")

In [ ]:
for (key, val) in six.iteritems(sh_feature):
    print(" ", key, ":", val)

In [ ]:
glcmFeatures = glcm.RadiomicsGLCM(orig_itk, cmb_itk, **settings)
glcmFeatures.enableAllFeatures()

print("These GLCM features will be calculated")
for f in glcmFeatures.enabledFeatures.keys():
    print(' ', f)
    print(getattr(glcmFeatures, 'get%sFeatureValue' % f).__doc__)
print('\nCalculating GLCM features..')
glcm_feature = glcmFeatures.execute()
print("Done")

In [ ]:
print("Here are the calculated GLCM features!")
for (key, val) in six.iteritems(glcm_feature):
    print(' ', key, ":", val)

In [ ]:
ngtdmFeatures = ngtdm.RadiomicsNGTDM(orig_itk, cmb_itk, **settings)
ngtdmFeatures.enableAllFeatures()

print("Now let's do NGTDM")
for f in ngtdmFeatures.enabledFeatures.keys():
    print(' ', f)
    print(getattr(ngtdmFeatures, 'get%sFeatureValue' % f).__doc__)
    
print('Calculating')
ngtdm_feature = ngtdmFeatures.execute()
print("done!")

In [ ]:
print("NGTDM feature extraction result")
for (key, val) in six.iteritems(ngtdm_feature):
    print(' ', key, ":", val)

<hr style="color:olive">
<hr style="color:olive">
Show filtered images... : I'm not sure what these are
<hr style="color:olive">
<hr style="color:olive">

In [ ]:
applyLog = True
applyWavelet = True

In [ ]:
if applyLog: # "Firstorder features calculated on LoG-filtered-image"
    sigmaValues = np.arange(5., 0., -.5)[::1]
    for logImage, imageTypeName, inputKwargs in imageoperations.getLoGImage(orig_itk, cmb_itk, sigma=sigmaValues):
        logFirstorderFeatures = firstorder.RadiomicsFirstOrder(logImage, cmb_itk, **inputKwargs)
        logFirstorderFeatures.enableAllFeatures()
        results = logFirstorderFeatures.execute()
        for (key, val) in six.iteritems(results):
            laplacianFeatureName = "%s_%s" % (imageTypeName, key)
            print(' ', laplacianFeatureName, ":", val)

In [ ]:
if applyWavelet: # same but with the wavelet filter
    for decompositionImage, decompositionName, inputKwargs in imageoperations.getWaveletImage(orig_itk, cmb_itk):
        waveletFirstOrderFeatures = firstorder.RadiomicsFirstOrder(decompositionImage, cmb_itk, **inputKwargs)
        waveletFirstOrderFeatures.enableAllFeatures()
        results = waveletFirstOrderFeatures.execute()
        print("Calculated firstorder features with wavelet ", decompositionName)
        for (key, val) in six.iteritems(results):
            waveletFeatureName = "%s_%s" % (str(decompositionName), key)
            print(" ", waveletFeatureName, ":", val)

<hr style="color:orange">
<hr style="color:orange">
OR DO ALL AT ONCE?
<hr style="color:orange">
<hr style="color:orange">

In [ ]:
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.disableAllFeatures()
extractor.enableAllFeatures()
# extractor.enableFeaturesByName(firstorder=["Mean", "Kurtosis", "Median", "Skewness", "Variance"])

In [ ]:
print("Calculating")
featureVector = extractor.execute(orig_itk, cmb_itk)

In [ ]:
for key in featureVector.keys():
    print(key)

In [ ]:
for featureName in featureVector.keys():
    if featureName.find("diagnostic")!=-1 or featureName.find("shape")!=-1 or featureName.find("glcm")!=-1 or featureName.find("first")!=-1 or featureName.find("ngtdm")!=-1:
        print("%s: %s" % (featureName, featureVector[featureName]))

<hr style="color:purple">
<hr style="color:purple">
Z-score standardization
<hr style="color:purple">
<hr style="color:purple">

In [ ]:
fo_list = []
for key in fo_result.keys():
    fo_list.append(float(fo_result[key]))
ngtdm_list = []
for key in ngtdm_feature.keys():
    ngtdm_list.append(float(ngtdm_feature[key]))
shape_list = []
for key in sh_feature.keys():
    shape_list.append(float(sh_feature[key]))
glcm_list = []
for key in glcm_feature.keys():
    glcm_list.append(float(glcm_feature[key]))

In [ ]:
def zscore(data):
    mean = np.mean(data)
    std = np.std(data)
    z_scores = [(x - mean) / std for x in data]
    return z_scores

In [ ]:
all_feature = fo_list + shape_list + glcm_list + ngtdm_list
all_feature = zscore(all_feature)
all_feature = torch.tensor(all_feature)
all_feature = all_feature.view(1, -1)
all_feature.shape

In [ ]:
all_feature

<hr style="color:red">
<hr style="color:red">
THE MODEL
<hr style="color:red">
<hr style="color:red">

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchsummary as ts

In [ ]:
# 할거 ㅇㅇ
class CMBWATCHDOG(nn.Module):
    def __init__(self):
        super(CMBWATCHDOG, self).__init__()
        self.conv3d = nn.Conv3d(in_channels=2, out_channels=2, kernel_size=2)
    def forward(self, x):
        x = self.conv3d(x)
        return x
model = CMBWATCHDOG()
ts.summary(model, (2,2,2))

<hr style="color:skyblue">
<hr style="color:skyblue">
Voxel-based extraction with tqdm progress bar!
<hr style="color:skyblue">
<hr style="color:skyblue">

In [ ]:
from __future__ import print_function

In [ ]:
def tqdmProgressBar():
    global extractor
    radiomics.setVerbosity(logging.INFO) # verbose at least INFO
    import tqdm
    radiomics.progressReporter = tqdm.tqdm
def clickProgressBar():
    global extractor
    extractor.enableFeatureClassByName("glcm")
    radiomics.setVerbosity(logging.INFO)
    import click
    class progressWrapper:
        def __init__(self, iterable, desc=""):
            self.bar = click.progressbar(iterable, label=desc)
        def __iter__(self):
            return self.bar.__iter__()
        def __enter__(self):
            return self.bar.__enter__()
        def __exit__(self, exc_type, exc_value, tb):
            return self.bar.__exit__(exc_type, exc_value, tb)
    radiomics.progressReporter = progressWrapper

In [ ]:
logger.setLevel(logging.DEBUG)

In [ ]:
extractor2 = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)
featureClasses = getFeatureClasses()

In [ ]:
tqdmProgressBar()

In [ ]:
print("Active features:")
for cls, features in six.iteritems(extractor2.enabledFeatures):
    if features is None or len(features) == 0 :
        features = [f for f, deprecated in six.iteritems(featureClasses[cls].getFeatureNames()) if not deprecated]
    for f in features:
        print(f)
        print(getattr(featureClasses[cls], 'get%sFeatureValue' % f).__doc__)

In [ ]:
print("Calculating features...")
featureVector2 = extractor2.execute(orig_itk, cmb_itk, voxelBased=True) ## THIS IS THE DECIDING PARAMETER...

In [ ]:
### THINK AGAIN BEFORE RUNNING THIS CELL! THIS WILL OUTPUT A LOT OF FILES UNDER PWD
for featureName, featureValue in six.iteritems(featureVector2):
    if isinstance(featureValue, sitk.Image):
        sitk.WriteImage(featureValue, "%s_%s.nrrd" % ("SWI", featureName))
        print("%s, stored as '%s_%s.nrrd'" % (featureName, "SWI", featureName))
    else:
        print("%s: %s" % (featureName, featureValue))

<hr style="color:red">
<hr style="color:red">
RESAMPLING?? -> NOPE DON'T DO THIS
<hr style="color:red">
<hr style="color:red">

In [ ]:
(ii, im) = imageoperations.resampleImage(orig_itk, cmb_itk, resampledImageSpacing=[2, 2, 2], resampledPixelSpacing=[2, 2, 2], label=1, padDistance=5)

In [ ]:
sitk.WriteImage(ii, "./resampled/21102001/orig_resamp_.nii.gz")
sitk.WriteImage(im, "./resampled/21102001/cmb_resamp_.nii.gz")
# this process crops the region that contains a group of ROI? 
# unnecessary

<hr style="color:pink;width:50px;margin-left:0px" align="left">
<hr style="color:pink;width:50px;margin-left:10px" align="left">
<hr style="color:pink;width:50px;margin-left:20px" align="left">
<hr style="color:pink;width:50px;margin-left:30px" align="left">
<hr style="color:pink;width:50px;margin-left:40px" align="left">
BELOW IS FOR IMAGE DISPLAYING

In [ ]:
%matplotlib inline
from monai.transforms import LoadImaged, EnsureChannelFirstd, ResampleToMatchd, Orientationd
from matplotlib import animation, rc; rc("animation", html="jshtml")
import gc

transform = Compose(
    [
        LoadImaged(reader=("PydicomReader", "nibabelreader"), keys=["image", "seg"]),
        EnsureChannelFirstd(keys=["image", "seg"]),
        Orientationd(keys=["image", "seg"], axcodes="RAS"),
    ]
)

def create_animation(img, seg, seg_rev=False, fps=10):
    images = img
    segs = seg
    if seg_rev:
        segs = segs[::-1]
    ims_sgs = [np.concatenate([images[i], segs[i]], axis=1) for i in range(len(images))]
    animation_arr = np.stack(ims_sgs, axis=0)
    del images, ims_sgs
    gc.collect()
    
    fig = plt.figure(figsize=(5,5), dpi=160)
    im = plt.imshow(animation_arr[0], cmap="bone")
    plt.axis("off")

    def animate_func(i):
        im.set_array(animation_arr[i])
        return [im]
    plt.close()
    
    anim = animation.FuncAnimation(fig, animate_func, frames=animation_arr.shape[0], interval=1000//fps)
    return anim

In [ ]:
data = {"image":orig_list[0], "seg":cmb_list[0]}
output = transform(data)
img = output["image"].numpy().transpose([0, 3, 2, 1])[0]
seg = output["seg"].numpy().transpose([0, 3, 2, 1])[0]
img = (img-np.min(img))/(np.max(img)-np.min(img)+1e-6)
img = (img*255).astype(np.uint8)
seg = np.where(seg>0, 255, 0).astype(np.uint8)

create_animation(img, seg, fps=30)

IMAGE DISPLAY ENDS
<hr style="color:pink;width:50px;margin-left:40px" align="left">
<hr style="color:pink;width:50px;margin-left:30px" align="left">
<hr style="color:pink;width:50px;margin-left:20px" align="left">
<hr style="color:pink;width:50px;margin-left:10px" align="left">
<hr style="color:pink;width:50px;margin-left:0px" align="left">